# <font color='darkgreen'>บทที่ 1 - Machine Learning คืออะไร?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-builders/curriculum/blob/main/notebooks/01_ml_what.ipynb)

ในบทเรียนนี้เราจะเรียนรู้ว่า Artificial Intelligence (AI), Machine Learning (ML) และ Deep Learning (DL) คืออะไร เหมือนกันหรือแตกต่างกันอย่างไร เราจะเรียนรู้ส่วนประกอบของระบบ machine learning และวิธีการเทรน machine learning model ด้วยตัวอย่างจำแนกรูปภาพอาหารไทย 48 ชนิดจากชุดข้อมูล [FoodyDudy](https://github.com/GemmyTheGeek/FoodyDudy) หลังจากนั้นเราจะเห็นว่าส่วนประกอบและวิธีการเทรนนี้ถูกใช้กับข้อมูลชนิดอื่นๆ เช่น ข้อความ (texts) และตาราง (tabular data) ได้อย่างไรบ้าง

Checkpoint ท้ายบทประกอบด้วยคำถามชวนคิดเกี่ยวกับบทเรียน

บทเรียนนี้ปรับแต่งและเพิ่มเติมจาก [fastai Practical Deep Learning for Coders v4 part1 - Lesson 1](https://course.fast.ai/videos/?lesson=1) และ [โครงการ AI Builders](https://github.com/ai-builders/curriculum)



## <font color='saddlebrown'>ติดตั้ง Package ที่ต้องใช้งาน

In [ ]:
#ลง package ที่ต้องใช้; รันแค่ครั้งเดียวก็พอ
!pip install pydantic
!pip install scikit-image
!pip install pandas
!pip install statsmodels
!pip install torch
#pytorch ignite
!pip install -q --pre pytorch-ignite==0.5.0.dev20230325

#fastai
!pip install -q fastbook==0.0.29

#autogluon
!pip install --upgrade -q mxnet==1.9.1
!pip install -q autogluon==0.7.0

#pythainlp
#!pip install -q pythainlp==3.1.1
!pip install pythainlp
#transformers
#!pip install -q transformers==4.27.3
!pip install transformers
exit() #reset runtime เพื่อให้ package ใช้งานได้อย่างถูกต้อง

## <font color='saddlebrown'>เริ่มทำ ML กัน! Image Classification with fastai and Resnet

ผลงาน [โมเดลจำแนกรูปอาหารไทย 48 ประเภท](https://www.facebook.com/groups/dsbkkgroup/posts/1733262050179128/) โดย [@GemmyTheGeek](https://github.com/GemmyTheGeek) ศิษย์เก่าโครงการ AI Builders (Class of 2021) เราจะเทรน ML model ด้วยข้อมูลรูปภาพอาหารไทยขนาด 224 x 224 pixels จำนวน 240 รูปต่อเมนูอาหาร (รวม 240 x 48 = 11,520 รูป) และทดสอบบน mini validation set ที่มีข้อมูล 5 รูปต่อเมนูอาหาร (รวม 5 x 48 = 240 รูป) ว่ามีความแม่นยำ (accuracy) แค่ไหน

โมเดลที่ดีที่สุดของ [@GemmyTheGeek](https://github.com/GemmyTheGeek) มีความแม่นยำถึง 97% หลังผ่านการปรับแต่ง สามารถทดลองใช้ได้[ที่นี่](https://huggingface.co/spaces/cstorm125/foodydudy_for_lesson1/)

### FoodyDudy - ชุดข้อมูลอาหารไทย 48 ประเภท

In [ ]:
#download รูปอาหารจาก FoodyDudy
!git clone https://github.com/GemmyTheGeek/FoodyDudy.git
!rm -r FoodyDudy/images/valid_mini
!mkdir FoodyDudy/images/valid_mini

#สร้าง valid mini เพื่อความเร็วในการทำ validation
import glob
import shutil
import os

valid_folders = glob.glob('FoodyDudy/images/valid/*')
valid_fnames = []
for folder in valid_folders:
    valid_fnames += glob.glob(f'{folder}/*')[:5]
for i in range(48):
    os.mkdir(f'FoodyDudy/images/valid_mini/{str(i).zfill(2)}')
for fname in valid_fnames:
    shutil.copyfile(fname, f'FoodyDudy/images/valid_mini/{fname.split("/")[-2]}/{fname.split("/")[-1]}')

<img src=https://github.com/GemmyTheGeek/FoodyDudy/raw/main/47thaifood.png width="500px">


In [ ]:
#เปลี่ยน label จากตัวเลขเป็นชื่อภาษาอังกฤษ เพื่อความง่ายในการดูผล
food_d = [
 'green_curry', 'tepo_curry', 'liang_curry', 'taohoo_moosup', 'mara_yadsai',
 'masaman', 'orange_curry', 'cashew_chicken', 'omelette', 'sunny_side_up',
 'palo_egg', 'sil_egg', 'nun_banana', 'kua_gai', 'cabbage_fish_sauce',
 'river_prawn', 'shrimp_ob_woonsen', 'kanom_krok', 'mango_sticky_rice', 'kao_kamoo',
 'kao_klook_kapi', 'kaosoi', 'kao_pad', 'kao_pad_shrimp', 'chicken_rice',
 'kao_mok_gai', 'tom_ka_gai', 'tom_yum_kung', 'tod_mun', 'poh_pia',
 'pak_boong_fai_daeng', 'padthai', 'pad_krapao', 'pad_si_ew', 'pad_fakthong',
 'eggplant_stirfry', 'pad_hoi_lai', 'foithong', 'panaeng', 'yum_tua_ploo',
 'yum_woonsen', 'larb_moo', 'pumpkin_custard', 'sakoo_sai_moo', 'somtam',
 'moopoing','satay', 'hor_mok'
]

for i in range(48):
    os.rename(f"FoodyDudy/images/train/{str(i).zfill(2)}", f"FoodyDudy/images/train/{str(i).zfill(2)}_{food_d[i]}")
for i in range(48):
    os.rename(f"FoodyDudy/images/valid_mini/{str(i).zfill(2)}", f"FoodyDudy/images/valid_mini/{str(i).zfill(2)}_{food_d[i]}")

### เทรนโมเดลด้วย fastai

[fastai](https://docs.fast.ai/) เป็น package สำหรับเทรน deep learning model ที่สร้างต่อยอดบน [Pytorch](https://pytorch.org/) ทำให้เราสามารถเทรน deep learning model ที่มีประสิทธิภาพสูงได้ง่ายและรวดเร็ว

In [ ]:
from fastbook import *

เนื่องจากเรามีข้อมูลรูปภาพกว่า 1 หมื่นรูป เราจึงไม่สามารถโหลดรูปภาพทั้งหมดเข้าไปใน memory ได้ในครั้งเดียว เราจึงต้องอาศัย dataloader เพื่อทำการโหลดข้อมูลเข้าไปทีละ batch ในที่นี้คือ batch ละ 64 รูป

`DataBlock` เป็น class สำหรับสร้าง dataloader เพื่อส่งรูปให้กับโมเดลของ fastai

In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #x - image; y - single class
    get_items=get_image_files, #get image
    splitter=GrandparentSplitter(valid_name='valid_mini'), #use parent folder as train-valid split
    get_y=parent_label, #use parent folder as label
    batch_tfms=aug_transforms(size=224)
    )
dls = dblock.dataloaders('FoodyDudy/images/', bs=64)

In [ ]:
dls.train.show_batch(max_n=9,nrows=3)

สร้าง trainer ชื่อ `learn` และโหลด pretrained weights ชื่อ `resnet34` เพื่อทำการเทรนโมเดล; pretrained weights คือ weights ที่ถูกเทรนเพื่อจำแนกรูปภาพสิ่งของ 1,000 ประเภทจากชุดข้อมูล [ImageNet](https://www.image-net.org/) กว่า 1.2 ล้านรูป เรานำโมเดลนี้มาปรับจูน (finetune) เพื่อใช้จำแนกรูปที่เราต้องการ เช่น รูปอาหารไทย 48 ชนิดแทน

วิธีการนี้เรียกว่า [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) กล่าวคือนำโมเดลที่ถูกเทรนเพื่อทำ task หนึ่ง (จำแนกรูปภาพสิ่งของทั่วไป, ทำนายคำต่อไปในประโยค ฯลฯ) มาปรับจูนเพื่อทำ task อื่นๆ (จำแนกรูปอาหารไทย, จำแนกชนิดประโยค ฯลฯ) เป็นเทคนิคที่นิยมใช้กันอย่างแพร่หลายใน deep learning โดยเฉพาะกับข้อมูลรูปภาพและข้อความ

เราเทรนโมเดล (เฉพาะส่วนจำแนกรูปภาพอาหารไทย) ด้วยรูปทั้งหมด 1 ครั้ง (1 epoch) ด้วย learning rate = 0.002 เราจะวัดผลด้วย metric คือ accuracy (ความแม่นยำ; ถ้าทายถูกได้ 1 ทายผิดได้ 0 และหาค่าเฉลี่ย) ได้ความแม่นยำกว่า 75% ในเวลาไม่ถึง 3 นาทีซึ่งถือว่าไม่เลวเลยสำหรับการจำแนกรูปภาพ 48 ประเภท (ถ้าเดาสุ่มมีความแม่นยำ 1/48 = 2.08%)

ศัพท์เทคนิคทั้งหมดนี้คุณอาจจะยังไม่ต้องเข้าใจทั้งหมดตอนนี้ แต่จะได้เรียนรู้ในบทเรียนต่อๆไป

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(epochs=0, freeze_epochs=1, base_lr=2e-3)

ดูผลการทำนายคร่าวๆใน validation set ว่าทำนายผิดจากอะไรเป็นอะไรบ้าง

In [ ]:
learn.show_results() #true label - บน; prediction - ล่าง

ดู confusion matrix ของผลการทำนายใจ validation set ทั้งหมด `Actual` คือ label ที่แท้จริง `Predicted` คือ label ที่โมเดลทำนาย

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

Metric (ตัวชี้วัด) สำหรับ classification อื่นๆ เช่น [precision, recall, f1-score](https://en.wikipedia.org/wiki/Precision_and_recall)

In [ ]:
interp.print_classification_report()

## <font color='saddlebrown'>Artificial Intelligence (AI), Machine Learning (ML), Deep Learning (DL) คืออะไร-เหมือน/ต่างกันอย่างไร

AI มีนิยามที่กว้างและหลากหลายเป็นอย่างยิ่ง หากอ้างอิงหนังสือ [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/) จะหมายถึงระบบที่คิดหรือทำอย่างมนุษย์หรืออย่างมีเหตุผล หรืออาจจะกล่าวได้ว่าเป็นระบบที่คิดและทำอย่างมี "[สติปัญญา](https://en.wikipedia.org/wiki/Intelligence)" ซึ่งเป็นคำที่มีนิยามกว้างและหลากหลายอีกเช่นกัน

ในทางปฏิบัติของโครงการ AI Builders เราจะนิยาม AI ว่าเป็นระบบที่คิดและทำอย่างชาญฉลาด (โดยจงใจไม่ให้นิยามอันแน่ชัดกับความ "ชาญฉลาด" นี้) โดยอาศัย "กฎ" เช่น `ถ้าเกิด X แล้วควรจะทำ Y` กฎเหล่านี้จะถูกสร้างโดยมนุษย์กำหนดขึ้นเองหรือให้เครื่องจักรเรียนรู้ขึ้นเองจากข้อมูลก็ได้

ML คือ AI ชนิดหนึ่งเกิดขึ้นในกรณีที่กฎถูกเรียนรู้ขึ้นมาจากข้อมูลในอดีตด้วยเทคนิคทางสถิต เช่น จากการสังเกตข้อมูลซื้อขายของลูกค้าในอดีต เราค้นพบกฎว่า `ถ้าเป็นวันเสาร์-อาทิตย์ ยอดขายจะเพิ่มขึ้น` เป็นต้น

DL คือเทคนิค ML ที่ใช้ Neural Networks หลายชั้นเป็นสถาปัตยกรรมในการเรียนรู้กฎเหล่านี้

<img src=https://github.com/ai-builders/curriculum/raw/main/images/ai_ml_dl.png width="700px">

<img src=https://github.com/ai-builders/curriculum/raw/main/images/ai_definitions.png width="700px">

## <font color='saddlebrown'>ML ต่างจาก AI รูปแบบอื่นอย่างไร

เราอาจจะแบ่ง AI คร่าวๆเป็น 2 ประเภทคือ

1. **Rule-based Systems** ที่กฎ (Rules) ถูกกำหนดขึ้นโดยมนุษย์ เมื่อเราใส่ Input และ Rules ที่เรากำหนดเข้าไปคำนวณ (Computation) ก็จะได้ Output ออกมา

2. **Machine Learning Systems** ที่กฎถูกเรียนรู้ขึ้นมาจาก Input และ Output ในอดีต เช่น พฤติกรรมลูกค้าในอดีต, รูปภาพหรือข้อความที่ถูก label ด้วยมนุษย์มาก่อน, ราคาทรัพย์สินในอดีต เป็นต้น เพื่อให้ได้ Rules ขึ้นมา โดยส่วนใหญ่เรียก Rules นี้ว่า ML Models หลังจากนั้นจึงทำ Rules ไปใช้งานกับข้อมูลชุดใหม่ในรูปแบบเดียวกับ Rule-based Systems

<img src=https://github.com/ai-builders/curriculum/raw/main/images/rules_vs_ml.png width="700px">

## <font color='saddlebrown'>ML Model เรียนรู้ได้อย่างไร

ตัวอย่าง `Training Loop` การจำแนกรูป `ส้มตำ` หรือ `ไม่ส้มตำ`

1. เตรียมข้อมูล `Inputs` (ตัวอย่าง; รูปภาพ) และ `Labels` (`ส้มตำ` หรือ `ไม่ส้มตำ`) ใส่เข้าไปใน `Dataloader`; `Dataloader` ทำหน้าที่ส่งข้อมูลให้ทีละ batch (จำนวนตัวอย่างต่อ batch เรียกว่า batch size)
2. ส่งข้อมูล Inputs ใน batch นั้นไปคำนวนด้วย `Weights` เพื่อให้ได้ `Predictions`; `Weights` เปรียบเสมือนตัวเลขค่าสัมประสิทธิ์ในสมการ เช่น ในสมการ `y = 2.5x^2 - 0.5x + 0.2` ค่า `Weights` คือ 2.5, -0.5  และ 0.2 (`x` คือ `Inputs` และ `y` คือ `Predictions`)
3. คำนวณ `Loss` ของ batch ด้วย `Loss Function`; `Loss Function` มีหน้าที่หา "ความแตกต่าง" ระหว่าง `Predictions` (คำทำนายจากโมเดลว่ามีโอกาสเป็น `ส้มตำ` แค่ไหน) และ `Labels` (ความเป็นจริงว่าเป็น `ส้มตำ` หรือ `ไม่ส้มตำ`) ค่า `Loss` คือผลลัพธ์ ยิ่ง `Loss` สูงก็ยิ่งแปลว่าคำทำนายของเราผิดมาก-เยอะ
4. หลังจากได้ `Loss` ของแต่ละ batch แล้วเราก็คำนวณ `Gradients` ด้วยวิธี Backpropagation และนำ `Gradients` ไปให้ `Optimizer` ทำการ update `Weights`; การอัพเดทในที่นี้กล่าวง่ายๆคือการเปลี่ยนตัวเลขในสมการ เช่น จาก `y = 0.1x^2 - 0.1x + 0.1 เป็น `y` = 2.5x^2 - 0.5x + 0.2` เป็นต้น โดยการทำเช่นนี้โดยทั่วไปจะทำให้โมเดลมี `Weights` ที่ทำนายได้แม่นยำมากขึ้นและส่งผลให้ `Loss` ลดลงเรื่อยๆ

ขั้นตอนการกระทำ 1.-4. สำหรับข้อมูล 1 batch เรียกว่า 1 iteration; หากทำจนครบทุก batch ของข้อมูลที่มี 1 ครั้งเรียกว่า 1 epoch เช่น หากเรามีข้อมูล 100 รูป, batch size = 5 รูป, เราจะมีทั้งหมด 20 batches, 1 epoch ของเราจะประกอบด้วย 20 iterations เครื่องมือการเทรน ML สมัยใหม่ส่วนใหญ่จะมี class สำหรับช่วยทำให้ `Training Loop` สะดวกขึ้นเรียกว่า `Trainer` และ class สำหรับเก็บข้อมูลระหว่างเทรนเรียกว่า `Logger` เพิ่มมาอีกด้วย



คุณเห็นความแตกต่างอะไรบ้างใน `Training Loop` ด้านล่าง?

**ก่อนเทรน**

<img src=https://github.com/ai-builders/curriculum/raw/main/images/training_loop50.png width="700px">

**หลังเทรน**

<img src=https://github.com/ai-builders/curriculum/raw/main/images/training_loop70.png width="700px">

## <font color='saddlebrown'>Metric and Splits - โมเดลทำได้ดีแค่ไหน-แค่ไหนเรียกดี

ค่า `Loss` สามารถบอกได้ว่า `Predictions` "แตกต่าง" จาก `Labels` มากแค่ไหน แต่ในการใช้งานจริงคุณไม่สามารถไปบอกคนอื่นว่า "โมเดลจำแนกสิ่งของ 10 ชนิดของผมมี `Loss` อยู่ที่ 0.234" แล้วให้เขาเข้าใจว่ามันดีหรือไม่ดี เราจึงต้องมี `Metric` (ตัวชี้วัด) ที่เหมาะสมกับงาน (ภาษาทางเทคนิคนิยมเรียกว่า `task`) ต่างๆ เช่น หากเป็นการจำแนกรูปภาพสิ่งของ (`image classification task`) เราอาจจะใช้ `Metric` เป็น accuracy, [precision, recall, หรือ f1-score](https://en.wikipedia.org/wiki/Precision_and_recall) หากเป็นการทำนายตัวเลข (`regression task`) เราอาจจะใช้ [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error), [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error), หรือ [mean absolute percentage error](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)

<img src=https://github.com/ai-builders/curriculum/raw/main/images/train_valid_test_splits.png width="500px">

โดยทั่วไปแล้ว เราจะแบ่งชุดข้อมูลเป็น 3 splits และคำนวณค่า metric บน `validation` และ `test` sets

1. `train` - ชุดข้อมูลที่ป้อนให้โมเดลเพื่อทำการเรียนรู้ตาม `Training Loop` ("แบบฝึกหัด")
2. `validation` - ชุดข้อมูลสำหรับคำนวณ metric เพื่อเลือกโมเดลที่ดีที่สุด เพราะในความเป็นจริงเราอาจจะเทรนโมเดลหลากหลายสถาปัตยกรรมหรือ hyperparameters; metric ที่ถูกคำนวณบน `validation` จึงใช้สำหรับเปรียบเทียบโมเดลที่ดีที่สุด ("ข้อสอบ pre-test จากโรงเรียนกวดวิชา")
3. `test` - ชุดข้อมูลสำหรับคำนวณ metric ของโมเดลที่เราเลือกแล้วว่าดีที่สุด เปรียบเสมือนตัวชี้วัดว่าโมเดลของเราจะทำได้ดีแค่ไหนในโลกแห่งความเป็นจริง; ชุดข้อมูลนี้จะถูกรันเพียงครั้งเดียวเท่านั้นในการทำโครงงาน ด้วยโมเดลที่ดีที่สุดที่เราเลือกมาจาก `validation` ("ข้อสอบจริง")

เหตุผลที่เราแบ่งชุดข้อมูล `train` กับ `validation` ออกจากกันเนื่องจากหากโมเดลได้รับการเทรนด้วยตัวอย่างใดตัวอย่างหนึ่งแล้ว มีความเป็นไปได้สูงที่โมเดลจะ "จดจำคำตอบ" ของตัวอย่างนั้นๆได้ เปรียบเสมือนหากเรานำข้อสอบปลายภาคมาให้นักเรียนทำเป็นแบบฝึกหัด นักเรียนย่อมมีโอกาสทำได้ดีเกือบเต็มแทบทุกคน แต่ไม่ใช่เพราะนักเรียนได้เรียนรู้สิ่งที่เราต้องการ แต่แค่เพราะจำคำตอบได้จากแบบฝึกหัดนั่นเอง

เหตุผลที่เราแบ่งชุดข้อมูล `validation` กับ `test` ออกจากกันเนื่องจาก หากเราใช้ข้อมูลชุดเดียวกันในการทดสอบโมเดลหลายรูปแบบ เรามีโอกาสที่จะจงใจปรับจูนโมเดลให้เหมาะกับ "ข้อสอบจริง" ได้โดยตั้งใจหรือไม่ตั้งใจ เราจึงปรับจูนโมเดลของเราด้วย `validation` แล้วจึงนำโมเดลที่ดีที่สุดไป "สอบจริง" ด้วย `test`

เพราะฉะนั้นเวลาเราแบ่ง `train`, `validation` และ `test` splits เราต้องทำให้มั่นใจว่าไม่มีตัวอย่างเดียวกันหลุดไปใน set อื่น เช่น ในการทำ face recognition ไม่ควรมีรูปคนๆเดียวกันในมากกว่า 1 set, หากทำ time series forecasting เรียงข้อมูลจากเก่าสุดไปใหม่สุดใน `train`, `validation` และ `test` เพื่อไม่ให้โมเดลเห็นข้อมูลจากอนาคต, หากทำ product recommendation ไม่ควรมีข้อมูลของลูกค้าคนเดียวกันในมากกว่า 1 set เพราะจะทำให้เราเดาว่าเขาเป็นลูกค้าประเภทไหนได้ด้วยข้อมูลที่ไม่ควรเห็น ฯลฯ ทั้งนี้ขึ้นอยู่กับวิจารณญาณของผู้สร้างโมเดล



## <font color='saddlebrown'>Image Classification with Pytorch and Resnet

เรามาลองจำแนกรูปอาหารไทย 48 ประเภทกันอีกครั้ง โดยครั้งนี้เราจะค่อยๆสร้างส่วนประกอบต่างๆขึ้นเองด้วย [Pytorch](https://pytorch.org/) และ [Pytorch Ignite](https://pytorch.org/ignite/index.html)

In [ ]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.transforms as T
from torchvision import models

from ignite.engine import Engine, Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Accuracy, Loss

try:
    from tensorboardX import SummaryWriter
except ImportError:
    try:
        from torch.utils.tensorboard import SummaryWriter
    except ImportError:
        raise RuntimeError(
            "This module requires either tensorboardX or torch >= 1.2.0. "
            "You may install tensorboardX with command: \n pip install tensorboardX \n"
            "or upgrade PyTorch using your package manager of choice (pip or conda).")

### Metric

In [ ]:
probs = torch.tensor([[0.4, 0.6],
                      [0.1, 0.9],
                      [0.9, 0.1]], dtype=torch.float)
preds = probs.argmax(1)
targets = torch.tensor([0,1,0])

print(probs)
print(preds)
print(targets)

In [ ]:
#accuracy
(preds==targets).float().mean()

### Datasets and Dataloaders

In [ ]:
train_dir = 'FoodyDudy/images/train'
valid_dir = 'FoodyDudy/images/valid_mini'
batch_size = 256

#image augmentations; fastai ทำให้เราอัตโนมัติ แต่เราสามารถกำหนดเองได้ด้วยว่าเราอยากได้อะไรบ้าง
train_transform = T.Compose([
    T.Resize(224),
    T.TrivialAugmentWide(),
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

valid_transform = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

#dataset
train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
valid_ds = datasets.ImageFolder(valid_dir, transform=valid_transform)

#dataloader
train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=0, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, num_workers=0, shuffle=False)

classes = train_ds.classes
n_classes = len(classes)

In [ ]:
from PIL import Image
import glob

#helper functions from https://pytorch.org/vision/master/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols,
                            squeeze=False,)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

test_images = glob.glob('FoodyDudy/images/test/00/*')
i = np.random.randint(0,len(test_images))
orig_img = Image.open(test_images[i])

augmenter = T.TrivialAugmentWide()
imgs = [augmenter(orig_img) for _ in range(2)]
plot(imgs)

In [ ]:
#helper functions adapted from https://pytorch.org/vision/master/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
def plot_row(imgs, img_per_row = 3):
    num_cols = img_per_row
    num_rows = len(imgs) // num_cols
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        ax = axs[row_idx // num_cols, row_idx % num_cols]
        ax.imshow(row.permute(1,2,0))
        ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    plt.tight_layout()

sample_imgs = next(iter(train_dl))
plot_row(sample_imgs[0][:9])

### Architecture and Weights

เราเลือกใช้ Architecture ชื่อ [resnet](https://pytorch.org/vision/stable/models.html) รูปแบบที่มี 50 layers (`resnet50`) ที่ถูก pretrain (เทรนมาก่อนหน้า) ด้วยรูปสิ่งของทั่วไปกว่า 1 ล้านรูป ([ImageNet](https://www.image-net.org/))

In [ ]:
backbone = models.resnet50(pretrained=True)
backbone

### Loss

เราเลือกใช้ Loss Function ที่เรียกว่า [Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [ ]:
#ลองปรับ probs และ targets ดูว่าทำให้ loss เปลี่ยนไปอย่างไรบ้าง
probs = torch.tensor([[0.4, 0.6],
                      [0.1, 0.9],
                      [0.9, 0.1]], dtype=torch.float)
preds = probs.argmax(1)
targets = torch.tensor([0,1,0])

print(probs)
print(preds)
print(targets)

In [ ]:
loss_function = nn.CrossEntropyLoss()
loss_function(input = probs,
              target = targets).item()

In [ ]:
#accuracy
(preds==targets).float().mean()

### Optimizer

เราใช้ [AdamW](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html) เป็น Optimizer

In [ ]:
optimizer = torch.optim.AdamW(backbone.parameters(), lr=3e-4)
optimizer

### Architecture as Class

เราเปลี่ยน layer สุดท้ายของ resnet50 จากการทำนายสิ่งของ 1,000 ประเภทเป็นการทำนายอาหาร 48 ประเภท

In [ ]:
class FoodResNet(nn.Module):
    def __init__(self, n_classes=48):
        super(FoodResNet, self).__init__()
        # ใช้สถาปัตยกรรม resnet50; เปลี่ยน layer สุดท้ายเป็นการทำนายอาหาร 48 classes
        self.backbone = models.resnet50(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False

        #เพิ่งแค่ตรงนี้
        self.backbone.fc = torch.nn.Linear(self.backbone.fc.in_features, n_classes)

    def forward(self, x):
        preds = self.backbone(x)
        return preds

### Training Loop

นำทุกสิ่งเข้ามารวมกันแล้วเริ่มการเทรน!

#### Putting It All Together

In [ ]:
train_dir = 'FoodyDudy/images/train'
valid_dir = 'FoodyDudy/images/valid_mini'
batch_size = 512

#image augmentations
train_transform = T.Compose([
    T.Resize(224),
    T.TrivialAugmentWide(), #image augmentation จากงาน https://arxiv.org/abs/2103.10158
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

valid_transform = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    # T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

#dataset
train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
valid_ds = datasets.ImageFolder(valid_dir, transform=valid_transform)

#dataloader
train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=0, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, num_workers=0, shuffle=False)

classes = train_ds.classes
n_classes = len(classes)

In [ ]:
#ใส่ model และ input ใน gpu (ถ้ามี) ไม่ก็ cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#architecture
model = FoodResNet(n_classes=len(train_ds.classes)).to(device)

#loss
loss_function = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

#trainer
trainer = create_supervised_trainer(model, optimizer, loss_function, device=device)

#metric
val_metrics = {"accuracy": Accuracy(), "ce_loss": Loss(loss_function)}
evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

#### Logger

Logger สำหรับบันทึก Metric ระหว่างการเทรน

In [ ]:
#logger
writer = SummaryWriter(log_dir='logs')
train_log_interval = 1 #1 iteration = 1 batch = example จำนวนเท่า batch size
eval_log_interval = len(train_dl) #1 epoch = จำนวน example ทั้งหมด / batch size = วนครบทุก example 1 รอบ

pbar = tqdm(initial=0,
            leave=False,
            total=len(train_dl),
            desc=f"epoch {0} - loss: {0:.4f} - lr: {0:.4f}")

#ทุก X iteration, บันทึก training loss
@trainer.on(Events.ITERATION_COMPLETED(every=train_log_interval))
def log_training_loss(engine):
    lr = optimizer.param_groups[0]["lr"]
    pbar.desc = f"epoch {engine.state.epoch} - train loss: {engine.state.output:.4f} - lr: {lr:.4f}"
    pbar.update(train_log_interval)
    writer.add_scalar("training/loss", engine.state.output, engine.state.iteration)

#ทุก epoch, บันทึก training loss, accuracy
@trainer.on(Events.ITERATION_COMPLETED(every=eval_log_interval))
def log_training_results(engine):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    avg_loss = metrics["ce_loss"]
    tqdm.write(
        f"train results - epoch: {engine.state.epoch} avg accuracy: {avg_accuracy:.2f} avg loss: {avg_loss:.2f}"
    )
    writer.add_scalar("training/avg_loss", avg_loss, engine.state.iteration)
    writer.add_scalar("training/avg_accuracy", avg_accuracy, engine.state.iteration)

#ทุก epoch, บันทึก validation loss, accuracy
@trainer.on(Events.ITERATION_COMPLETED(every=eval_log_interval))
def log_validation_results(engine):
    evaluator.run(valid_dl)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    avg_loss = metrics["ce_loss"]
    tqdm.write(
        f"valid results - epoch: {engine.state.epoch} avg accuracy: {avg_accuracy:.2f} avg loss: {avg_loss:.2f}"
    )
    writer.add_scalar("valdation/avg_loss", avg_loss, engine.state.iteration)
    writer.add_scalar("valdation/avg_accuracy", avg_accuracy, engine.state.iteration)
    pbar.n = pbar.last_print_n = 0

#### Train the Model

In [ ]:
trainer.run(train_dl, max_epochs=3)
writer.close()

### Evaluation

ดู Log ของการเทรนด้วย `tensorboard`

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/ --host 0.0.0.0

## <font color='saddlebrown'>Text Classification with HuggingFace and WangchanBERTa

ใช้โมเดลที่ดีที่สุด ทำ NLP task ที่ง่ายที่สุดคือการแยกข้อความ social media ว่าเป็น positive, neutral, negative หรือ question ด้วยชุดข้อมูล [wisesight_sentiment](https://github.com/PyThaiNLP/wisesight-sentiment) ผล benchmark ที่ดีที่สุดของชุดข้อมูลนี้ทำโดย [Wangchanberta](https://medium.com/airesearch-in-th/wangchanberta-%E0%B9%82%E0%B8%A1%E0%B9%80%E0%B8%94%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%A1%E0%B8%A7%E0%B8%A5%E0%B8%9C%E0%B8%A5%E0%B8%A0%E0%B8%B2%E0%B8%A9%E0%B8%B2%E0%B9%84%E0%B8%97%E0%B8%A2%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B9%83%E0%B8%AB%E0%B8%8D%E0%B9%88%E0%B9%81%E0%B8%A5%E0%B8%B0%E0%B8%81%E0%B9%89%E0%B8%B2%E0%B8%A7%E0%B8%AB%E0%B8%99%E0%B9%89%E0%B8%B2%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%AA%E0%B8%B8%E0%B8%94%E0%B9%83%E0%B8%99%E0%B8%82%E0%B8%93%E0%B8%B0%E0%B8%99%E0%B8%B5%E0%B9%89-d920c27cd433) ที่ micro-averaged F1 ที่ 76.19

เพื่อประหยัดเวลาในตัวอย่างนี้เราจะเทรนด้วย training set เพียงแค่ 5,000 ตัวอย่างจากกว่า 20,000 ข้อความ

<img src=https://github.com/ai-builders/curriculum/raw/main/images/wangchanberta_results.png width="700px">



In [ ]:
"""
Preprocessing for thai2transformers
"""
from typing import Collection, Callable
from functools import partial
import re
import html
from pythainlp.tokenize import word_tokenize

_TK_UNK, _TK_REP, _TK_WREP, _TK_URL, _TK_END = "<unk> <rep> <wrep> <url> </s>".split()

SPACE_SPECIAL_TOKEN = "<_>"

# str->str rules
def fix_html(text: str) -> str:
    """
        List of replacements from html strings in `test`. (code from `fastai`)
        :param str text: text to replace html string
        :return: text where html strings are replaced
        :rtype: str
        :Example:
            >>> fix_html("Anbsp;amp;nbsp;B @.@ ")
            A & B.
    """
    re1 = re.compile(r"  +")
    text = (
        text.replace("#39;", "'")
        .replace("amp;", "&")
        .replace("#146;", "'")
        .replace("nbsp;", " ")
        .replace("#36;", "$")
        .replace("\\n", "\n")
        .replace("quot;", "'")
        .replace("<br />", "\n")
        .replace('\\"', '"')
        .replace(" @.@ ", ".")
        .replace(" @-@ ", "-")
        .replace(" @,@ ", ",")
        .replace("\\", " \\ ")
    )
    return re1.sub(" ", html.unescape(text))


def replace_url(text: str) -> str:
    """
        Replace url in `text` with TK_URL (https://stackoverflow.com/a/6041965)
        :param str text: text to replace url
        :return: text where urls  are replaced
        :rtype: str
        :Example:
            >>> replace_url("go to https://github.com")
            go to <url>
    """
    URL_PATTERN = r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
    return re.sub(URL_PATTERN, _TK_URL, text)


def rm_brackets(text: str) -> str:
    """
        Remove all empty brackets and artifacts within brackets from `text`.
        :param str text: text to remove useless brackets
        :return: text where all useless brackets are removed
        :rtype: str
        :Example:
            >>> rm_brackets("hey() whats[;] up{*&} man(hey)")
            hey whats up man(hey)
    """
    # remove empty brackets
    new_line = re.sub(r"\(\)", "", text)
    new_line = re.sub(r"\{\}", "", new_line)
    new_line = re.sub(r"\[\]", "", new_line)
    # brakets with only punctuations
    new_line = re.sub(r"\([^a-zA-Z0-9ก-๙]+\)", "", new_line)
    new_line = re.sub(r"\{[^a-zA-Z0-9ก-๙]+\}", "", new_line)
    new_line = re.sub(r"\[[^a-zA-Z0-9ก-๙]+\]", "", new_line)
    # artifiacts after (
    new_line = re.sub(r"(?<=\()[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    new_line = re.sub(r"(?<=\{)[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    new_line = re.sub(r"(?<=\[)[^a-zA-Z0-9ก-๙]+(?=[a-zA-Z0-9ก-๙])", "", new_line)
    # artifacts before )
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\))", "", new_line)
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\})", "", new_line)
    new_line = re.sub(r"(?<=[a-zA-Z0-9ก-๙])[^a-zA-Z0-9ก-๙]+(?=\])", "", new_line)
    return new_line


def replace_newlines(text: str) -> str:
    """
        Replace newlines in `text` with spaces.
        :param str text: text to replace all newlines with spaces
        :return: text where all newlines are replaced with spaces
        :rtype: str
        :Example:
            >>> rm_useless_spaces("hey whats\n\nup")
            hey whats  up
    """

    return re.sub(r"[\n]", " ", text.strip())


def rm_useless_spaces(text: str) -> str:
    """
        Remove multiple spaces in `text`. (code from `fastai`)
        :param str text: text to replace useless spaces
        :return: text where all spaces are reduced to one
        :rtype: str
        :Example:
            >>> rm_useless_spaces("oh         no")
            oh no
    """
    return re.sub(" {2,}", " ", text)

def replace_spaces(text: str, space_token: str = SPACE_SPECIAL_TOKEN) -> str:
    """
        Replace spaces with _
        :param str text: text to replace spaces
        :return: text where all spaces replaced with _
        :rtype: str
        :Example:
            >>> replace_spaces("oh no")
            oh_no
    """
    return re.sub(" ", space_token, text)


def replace_rep_after(text: str) -> str:
    """
    Replace repetitions at the character level in `text`
    :param str text: input text to replace character repetition
    :return: text with repetitive tokens removed.
    :rtype: str
    :Example:
        >>> text = "กาาาาาาา"
        >>> replace_rep_after(text)
        'กา'
    """

    def _replace_rep(m):
        c, cc = m.groups()
        return f"{c}"

    re_rep = re.compile(r"(\S)(\1{3,})")
    return re_rep.sub(_replace_rep, text)


def replace_wrep_post(toks: Collection[str]) -> Collection[str]:
    """
    Replace reptitive words post tokenization;
    fastai `replace_wrep` does not work well with Thai.
    :param Collection[str] toks: list of tokens
    :return: list of tokens where repetitive words are removed.
    :rtype: Collection[str]
    :Example:
        >>> toks = ["กา", "น้ำ", "น้ำ", "น้ำ", "น้ำ"]
        >>> replace_wrep_post(toks)
        ['กา', 'น้ำ']
    """
    previous_word = None
    rep_count = 0
    res = []
    for current_word in toks + [_TK_END]:
        if current_word == previous_word:
            rep_count += 1
        elif (current_word != previous_word) & (rep_count > 0):
            res += [previous_word]
            rep_count = 0
        else:
            res.append(previous_word)
        previous_word = current_word
    return res[1:]


# combine them together
def process_transformers(
    text: str,
    pre_rules: Collection[Callable] = [
        fix_html,
        rm_brackets,
        replace_newlines,
        rm_useless_spaces,
        replace_spaces,
        replace_rep_after,
    ],
    tok_func: Callable = word_tokenize,
    post_rules: Collection[Callable] = [replace_wrep_post],
) -> str:
    text = text.lower()
    for rule in pre_rules:
        text = rule(text)
    toks = tok_func(text)
    for rule in post_rules:
        toks = rule(toks)
    return "".join(toks)

"""
Metric for thai2transformers
"""
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support, classification_report
def classification_metrics(pred, pred_labs=False):
    labels = pred.label_ids
    preds = pred.predictions if pred_labs else pred.predictions.argmax(-1)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average="macro")
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds, average="micro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1_micro': f1_micro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'f1_macro': f1_macro,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'nb_samples': len(labels)
    }


In [ ]:
import pandas as pd
from datasets import load_dataset, load_metric, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
#load dataset
train_ds, valid_ds, test_ds = load_dataset('wisesight_sentiment', split=['train[:5000]','validation','test'])
dataset = DatasetDict({'train': train_ds, 'validation': valid_ds, 'test': test_ds})
dataset = dataset.map(lambda examples: {'labels': examples['category']}, batched=True)
num_labels = len(set(dataset['train']['labels']))
dataset

In [ ]:
#ตัวอย่างข้อความ
#0 positive; 1 neutral; 2 negative; 3 question
pd.DataFrame(dataset['train'].shuffle()[:10])[['labels','texts']]

In [ ]:
#ทำความสะอาดข้อความ
def clean_function(examples):
    examples['texts'] = process_transformers(examples['texts'])
    return examples

cleaned_dataset = dataset.map(clean_function)

In [ ]:
#ตัวอย่างข้อความที่ทำความสะอาดแล้ว
pd.DataFrame(cleaned_dataset['train'].shuffle()[:10])[['labels','texts']]

In [ ]:
#tokenize ข้อความ; เปลี่ยนจาก texts เป็น index
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')
tokenizer(cleaned_dataset['train'][0]['texts'])

In [ ]:
def encode_function(examples):
    return tokenizer(examples['texts'], max_length=416, truncation=True)
encoded_dataset = dataset.map(encode_function, batched=True)

In [ ]:
#โหลดโมเดล Wangchanberta ที่ถูก pretrained มาใช้สำหรับ sequence classification
num_labels = len(set(encoded_dataset['train']['labels']))

model = AutoModelForSequenceClassification.from_pretrained(
    'airesearch/wangchanberta-base-att-spm-uncased',
    num_labels=num_labels)

In [ ]:
#setup trainer
train_args = TrainingArguments(
    output_dir = 'wisesight_sentiment_wangchanberta',
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    warmup_steps = int(len(encoded_dataset['train']) * 1 // 8 * 0.1),
    weight_decay=1e-2,
    save_total_limit=1,
    metric_for_best_model='f1_micro',
    seed = 125
)

trainer = Trainer(
    model,
    train_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=classification_metrics
)

In [ ]:
#เริ่มเทรน
trainer.train()

In [ ]:
#ผลลัพธ์
preds  = trainer.predict(encoded_dataset['test'])
pd.DataFrame.from_dict(preds[2],orient='index')

## <font color='saddlebrown'>Text Classification with scikit-learn and Logistic Regression

ใช้โมเดลง่ายๆทำ NLP task ง่ายๆ แล้วลองดูว่าได้ผลต่างกันแค่ไหน

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from pythainlp.tokenize import word_tokenize

#ตัดคำด้วยพจนานุกรมของ PyThaiNLP
word_tokenize(dataset['train']['texts'][2])

In [ ]:
#เปลี่ยนข้อความเป็น bag-of-words features
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)

In [ ]:
x_train = tfidf.fit_transform(dataset['train']['texts'])
x_valid = tfidf.transform(dataset['validation']['texts'])
x_test = tfidf.transform(dataset['test']['texts'])
x_train,x_valid,x_test

In [ ]:
#label คือ 0 positive; 1 neutral; 2 negative; 3 question
enc = LabelEncoder()
y_train = enc.fit_transform(np.array(dataset['train']['labels']))
y_valid = enc.transform(np.array(dataset['validation']['labels']))
y_test = enc.transform(np.array(dataset['test']['labels']))
y_train.shape, y_valid.shape, y_test.shape

In [ ]:
#เทรนด้วย logistic regression; ไม่มีการ pretrain ใดๆ
model = LogisticRegression(penalty='l2', C=1.0)
model.fit(x_train, y_train)

In [ ]:
#ได้ผลดีกว่า Wangchanberta ที่เทรนแค่ 1 epoch อีก!
#เรื่องนี้สอนให้รู้ว่า บางครั้งเราไม่ต้องใช้โมเดลที่ดีที่สุดเพื่อทำ task ง่ายๆก็ได้
#โมเดลที่ซับซ้อนที่สุดอาจจะทำให้เราได้ผลดีที่สุด แต่ก็ย่อมใช้เวลาและทรัพยการมากกว่าเช่นกัน
class Preds:
    label_ids = y_test
    predictions = model.predict_proba(x_test)

pd.DataFrame.from_dict(classification_metrics(Preds),orient='index').transpose()

## <font color='saddlebrown'>Tabular Data Classification with scikit-learn and Gradient Boosting

จำแนกการตอบสนองของลูกค้ากับแคมเปญที่บริษัทจัดด้วยชุดข้อมูล [Retail Transaction Data](https://www.kaggle.com/regivm/retailtransactiondata) จัดทำโดย [Tanat Iempreedee](https://th.linkedin.com/in/tanat-iempreedee-67001947) ดูตัวอย่างการวิเคราะห์เต็มๆได้ที่ [Campaign Response Model - Example.ipynb](https://colab.research.google.com/drive/1eTVlSYcwUYkYDW6sPTY8XVO1QMhr-DNT?usp=sharing)

In [ ]:
#download data
!wget https://github.com/ai-builders/curriculum/raw/main/data/Retail_Data_Transactions.csv
!wget https://github.com/ai-builders/curriculum/raw/main/data/Retail_Data_Response.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
#จัดการข้อมูลให้อยู่ในรูปแบบ (X, y)
df_response = pd.read_csv('Retail_Data_Response.csv')
df_trans = pd.read_csv('Retail_Data_Transactions.csv', parse_dates=['trans_date'])

campaign_date = datetime(2015,3,17)
df_trans['age'] = (campaign_date - df_trans.trans_date).dt.days

In [ ]:
df = df_trans.groupby('customer_id').agg(
    recency=('age', 'min'),
    frequency=('customer_id', 'count'),
    monetary=('tran_amount', 'sum'),
    tenure=('age', 'max'),
    length_of_stay=('age', lambda x: x.max() - x.min())
)
df['ticket_size'] = df.monetary / df.frequency
df['std_ticket_size'] = df_trans.groupby('customer_id').tran_amount.std()
df['cv_ticket_size'] = df.std_ticket_size / df.ticket_size

# the number of months visited
active_months = df_trans.groupby(['customer_id', pd.Grouper(key='trans_date',freq='M')]).size().reset_index()
active_months = active_months.groupby('customer_id').trans_date.nunique()
df['active_months'] = active_months
df['avg_spend_m'] = df.monetary / df.active_months
df['avg_visit_m'] = df.frequency / df.active_months
# done with active_months, then drop it
df = df.drop(columns='active_months')
df['avg_tte'] = df.length_of_stay / (df.frequency - 1)

In [ ]:
#หน้าตาชุดข้อมูลตาราง
all_data = df_response.join(df, how='inner', on='customer_id').set_index('customer_id')
print(all_data.shape)
all_data.head()

In [ ]:
#split ข้อมูลเป็น train, validation และ test
random_state = 1024
test_size = 0.2
val_size = 0.2

X = all_data.drop(columns='response')
y = all_data.response

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_size,
                                                            stratify=y, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size/(1-test_size),
                                                  stratify=y_train_val, random_state=random_state)

train_data = X_train.join(y_train)
val_data = X_val.join(y_val)
test_data = X_test.join(y_test)

print(f"Train:      {len(train_data)} {len(train_data)/len(all_data):.3%}")
print(f"Validation: {len(val_data)} {len(val_data)/len(all_data):.3%}")
print(f"Test:       {len(test_data)} {len(test_data)/len(all_data):.3%}")

In [ ]:
X_val.shape, X_train.shape

In [ ]:
#สร้างโมเดลต้นไม้ด้วย xgboost (gradient boosted trees)
import xgboost as xgb

xgb_params = {'objective': 'binary:logistic',
              'random_state': 1024,
              'eval_metric': 'auc',
              'early_stopping_rounds': 10,
              'scale_pos_weight' : (len(y_train) - sum(y_train)) / sum(y_train),
              'max_depth': 3,
              'booster':'gbtree',
              }

model = xgb.XGBClassifier(**xgb_params)
model = model.fit(X_train,
                  y_train,
                  eval_set=[(X_val,y_val)],
                  verbose=False)

In [ ]:
#รายงานผล
print(classification_report(y_test,model.predict_proba(X_test)[:,1].round()))

In [ ]:
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

## <font color='saddlebrown'>Tabular Data Classification with AutoGluon Baselines

ใช้ package สำหรับทำ AutoML (ทดลอง models และ hyperparameters หลากหลายให้เองโดยไม่ต้องใช้คนทำ) สำหรับปัญหาเดียวกัน

ไปทดลองใช้ [AutoGluon](https://auto.gluon.ai/) เพิ่มเติมกันได้ ใช้ได้ไม่เฉพาะกับข้อมูลตารางเท่านั้นแต่ได้กับข้อมูลรูปและข้อความด้วย

In [ ]:
#load autogluon library
from autogluon.tabular import TabularPredictor

In [ ]:
#สร้างโมเดล
predictor = TabularPredictor(label='response',
                             eval_metric='f1_micro').fit(train_data=train_data)

In [ ]:
#รายงานผล
#ในกรณีนี้ การใช้โมเดลหลายๆโมเดลมาช่วยกันทำนาย (ensemble) ทำให้ได้ผลดีขึ้นมาก
predictor.leaderboard(test_data, silent=True)

# <font color='darkgreen'>Checkpoint ท้ายบท

## คำถามชวนคิดเกี่ยวกับบทเรียน

1. รอบตัวของคุณมีอะไรที่เป็น AI แบบ 1) Rule-based Systems และ 2) Machine Learning Systems บ้าง จงยกตัวอย่างมาอย่างละ 3 อัน

2. คุณได้เรียนรู้ส่วนประกอบต่างๆของ machine learning model คุณคิดว่าส่วนประกอบไหนมีความสำคัญที่สุดเพื่อให้ได้ผลลัพธ์การทำนายที่ดีที่สุดระหว่าง `Inputs/Labels` (ชุดข้อมูล), `Weights` (ตัวโมเดล), `Loss Function`, `Optimizer` และเพราะอะไร

3. ในโครงงานที่คุณอยากจะทำ คุณคิดว่าจะแบ่ง train-validation-test splits อย่างไร เพราะอะไร

4. คุณจะเห็นได้ว่าการเทรน machine learning model สามารถทำได้ด้วยเครื่องมือหลากหลายชนิด แต่ละเครื่องมือก็มี 1) ความยากง่ายในการใช้งาน 2) ความสามารถในการปรับแต่ง 3) ความมีประสิทธิภาพ ที่แตกต่างกัน 4) เหมาะกับข้อมูลรูปแบบต่างกัน (images, texts, tabular data) คุณคิดว่าเราควรเลือกใช้เครื่องมือเหล่านี้อย่างไรเมื่อคำนึงถึง 1)-4)





## <font color='saddlebrown'>ทบทวนคำศัพท์</font>

1. **<font color='darkblue'>`Inputs`</font>** (ชื่อเรียกอื่น: Features, Independent Variables, "X") - ข้อมูลที่ใส่เข้าไปในโมเดลเพื่อทำการเทรน/เพื่อให้ได้ `Predictions`

2. **<font color='darkblue'>`Labels`</font>** (ชื่อเรียกอื่น: Targets, Dependent Variables, Ground Truth, "y_true") - ข้อมูลที่โมเดลกำลังพยายามทำนาย

3. **<font color='darkblue'>`Predictions`</font>** (ชื่อเรียกอื่น: Outputs, "y_predict") - ผลการทำนายจากโมเดล ส่วนใหญ่สำหรับ classification จะเป็น probabilities สำหรับแต่ละ label สำหรับ regression จะเป็นจำนวนจริง

4. **<font color='darkblue'>`Weights`</font>** (ชื่อเรียกอื่น: Parameters, Coefficients, Models, "W, b") - ค่าของฟังชั่นสำหรับเปลี่ยน `Inputs` เป็น `Predictions`; รูปแบบของ `Weights` (รูปแบบสมการ) มีหลากหลายชนิด เรียกว่า `Architecture` หลายครั้งเวลาพูดถึง ML models เราหมายถึงส่วน `Weights` ที่ถูกเทรนด้วยข้อมูลเรียบร้อยแล้วนั่นเอง

5. **<font color='darkblue'>`Loss Function`/`Loss`</font>** (ชื่อเรียกอื่น: Cost Function, Objective Function; แต่ Objective Function จะอยากให้เยอะหรือน้อยก็ได้) - ฟังชั่นสำหรับเปลี่ยน `Labels` และ `Predictions` ให้เป็นค่า `Loss`; `Loss` ยิ่งน้อยยิ่งแสดงว่าโมเดลทำได้ดี

6. **<font color='darkblue'>`Gradients`</font>** - ค่า partial derivative ของ `Loss` ด้วย `Weights`

7. **<font color='darkblue'>`Optimizer`</font>** - class สำหรับทำการ update `Weights` ด้วย `Gradients` (อาจจะเป็นการบวก-ลบ-คูณ-หาร `Weights` ปัจจุบันด้วย `Gradients`)

8. **<font color='darkblue'>`Training Loop`</font>** - การส่ง `Inputs` และ `Labels` ให้โมเดลเรียนรู้ด้วยการ update `Weights`

9. **<font color='darkblue'>`Example`</font>** - ตัวอย่างในการเทรนโมเดล โดยทั่วไปประกอบด้วยคู่ `Inputs` และ `Labels` (ในกรณี ML ชนิดที่เราสนใจคือ [supervised learning](https://en.wikipedia.org/wiki/Supervised_learning)

10. **<font color='darkblue'>`Batch`/`Batch Size`</font>** - `Example` จำนวน X อัน โดย X คือ `Batch Size`

11. **<font color='darkblue'>`Dataloader`</font>** - class สำหรับส่ง `Batch` ไปให้โมเดลเทรน ในบางกรณี เราจำเป็นต้องส่งข้อมูลไปทีละ `Batch` เรื่องจากข้อมูลทั้งหมดใหญ่เกินที่จะใส่เข้าไปใน memory ของเครื่อง

12. **<font color='darkblue'>`Iteration`</font>** - การที่โมเดลเรียนรู้ 1 `Batch` จนครบ `Training Loop`

13. **<font color='darkblue'>`Epoch`</font>** - การที่โมเดลเรียนรู้จาก `Example` ทั้งหมด 1 รอบ

14. **<font color='darkblue'>`Trainer`</font>** - class สำหรับจัดการ `Training Loop`

15. **<font color='darkblue'>`Logger`</font>** - class สำหรับจดบันทึกข้อมูลการเทรน

16. **<font color='darkblue'>`Metric`</font>** - ตัวชี้วัดว่าโมเดลทำงานได้ดีแค่ไหน โดยส่วนใหญ่เป็นคนละฟังชั่นกับ `Loss Function`

17. **<font color='darkblue'>`Train Set`</font>** - ชุดข้อมูลที่ป้อนให้โมเดลเพื่อทำการเรียนรู้ตาม `Training Loop` ("แบบฝึกหัด")

18. **<font color='darkblue'>`Validation Set`</font>** (ชื่อเรียกอื่น: Development Set, Dev Set) - ชุดข้อมูลสำหรับคำนวณ metric เพื่อเลือกโมเดลที่ดีที่สุด เพราะในความเป็นจริงเราอาจจะเทรนโมเดลหลากหลายสถาปัตยกรรมหรือ hyperparameters; metric ที่ถูกคำนวณบน `validation` จึงใช้สำหรับเปรียบเทียบโมเดลที่ดีที่สุด ("ข้อสอบ pre-test จากโรงเรียนกวดวิชา")

19. **<font color='darkblue'>`Test Set`</font>** - ชุดข้อมูลสำหรับคำนวณ metric ของโมเดลที่เราเลือกแล้วว่าดีที่สุด เปรียบเสมือนตัวชี้วัดว่าโมเดลของเราจะทำได้ดีแค่ไหนในโลกแห่งความเป็นจริง; ชุดข้อมูลนี้จะถูกรันเพียงครั้งเดียวเท่านั้นในการทำโครงงาน ด้วยโมเดลที่ดีที่สุดที่เราเลือกมาจาก `validation` ("ข้อสอบจริง")